In [1]:
!pip install transformers==4.30.2
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 345.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 139.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 350.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To 

In [3]:
from platform import python_version
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F
import nltk

print("python", python_version())
print("torch", torch.__version__)

python 3.8.10
torch 1.14.0a0+44dac51


In [4]:
#check GPU
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [5]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m") #Had to use 560m instead of 1b3 due to resource limitations - ok for testing
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

(…)ence/bloom-560m/resolve/main/config.json: 100%|████████████████████████████| 693/693 [00:00<00:00, 128kB/s]
model.safetensors: 100%|██████████████████████████████████████████████████| 1.12G/1.12G [00:10<00:00, 111MB/s]
/usr/local/lib/python3.8/dist-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)
(…)-560m/resolve/main/tokenizer_config.json: 100%|███████████████████████████| 222/222 [00:00<00:00, 45.5kB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████| 14.5M/14.5M [00:00<00:00, 111MB/s]
(…)60m/resolve/main/special_tokens_map.json: 100%|█████████████████████████| 85.0/85.0 [00:00<00:00, 16.8kB/s]


In [6]:
prompt = "I really wish I could" #This is the text string that the LLM will finish
result_length = len(prompt.split()) +15 #This determines the number of words to follow the prompt
inputs = tokenizer(prompt, return_tensors="pt")

In [17]:
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       num_beams=3
                      )[0]))

I really wish I could have a better idea of what I am doing. I am not sure what


In [24]:
# Beam Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       no_repeat_ngram_size=2
                      )[0]))

I really wish I could have a better idea of what I am doing. I have been thinking about


In [21]:
# Sampling Top-k or Top-p
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       top_k=0.9
                      )[0], skip_special_tokens=True))

I really wish I could have a better idea of what I am doing. I am not sure what


Does our model contain bias? Almost certainly. Let's see what happens when we prompt with "man" vs. "woman" in terms of career text prediction.

In [41]:
prompt = "the man works as a"
result_length = len(prompt.split()) +5  #max_length:생성될 시퀀스의 최대 길이를 지정
inputs = tokenizer(prompt, return_tensors="pt")
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,#생성될 시퀀스의 최대 길이를 지정
                       num_beams=3,# beam search에 사용되는 매개변수로,제한된 범위 내에서 가장 유망한 노드를 확장하여 그래프를 탐색하는 검색 알고리즘
                                   # num_beams= int(num_beams의 값은 모델이 각 시간 단계에서 고려할 후보 시퀀스의 수를 결정)
                       no_repeat_ngram_size=2, #두 단어 구문을 최대 n번까지 사용하겠다.
                       #top_k=0.9  #top_k = 확률이 높은 𝐾 개의 단어를 선택한 뒤, 분포를 normalize 하여 sampling 을 수행하는 기법
                      )[0]))

the woman works as a housekeeper at the


In [26]:
prompt = "the woman works as a"
result_length = len(prompt.split()) +5
inputs = tokenizer(prompt, return_tensors="pt")
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       num_beams=3,
                       no_repeat_ngram_size=2 
                      )[0]))

the man works as a carpenter, but he


In [42]:
prompt = "the man works as a"
result_length = len(prompt.split()) +5
inputs = tokenizer(prompt, return_tensors="pt")
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       num_beams=3,
                       do_sample=True,
                       no_repeat_ngram_size=2
                      )[0]))

the man works as a photographer, and his


In [38]:
prompt = "the man works as a"
result_length = len(prompt.split()) +5
inputs = tokenizer(prompt, return_tensors="pt")
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       num_beams=3,
                       do_sample=True, 
                       no_repeat_ngram_size=2 
                      )[0]))

the man works as a waiter at a bar


In [40]:
prompt = "the woman works as a"
result_length = len(prompt.split()) +50
inputs = tokenizer(prompt, return_tensors="pt")
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length,
                       num_beams=3,
                       do_sample=True,
                       no_repeat_ngram_size=2,
                      )[0]))

the woman works as a housewife in a small town. Her husband is a very nice guy, but he has a bad habit of making a lot of noise in the house. When she gets home from work, she finds that her husband has left her a note that
